In [1]:
library('SelectSim')
library('tidyverse')
library('tictoc')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] tictoc_1.2        lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

# MSK P vs M gene frequency analysis

In [3]:
msk_p_sample_run_list <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/primary_met_work/data/msk_p_sample_run_data_list.rds')
msk_m_sample_run_list <- readRDS('/mnt/ndata/arvind/co_mutation_project/analysis/primary_met_work/data/msk_m_sample_run_data_list.rds')

In [4]:
str(msk_p_sample_run_list[[1]])

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:306, 1:8195] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  .. .. .. ..$ : chr [1:8195] "GENIE-MSK-P-0038488-T01-IM6" "GENIE-MSK-P-0019305-T01-IM6" "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0069371-T01-IM7" ...
  .. ..$ truncating: num [1:306, 1:8195] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  .. .. .. ..$ : chr [1:8195] "GENIE-MSK-P-0038488-T01-IM6" "GENIE-MSK-P-0019305-T01-IM6" "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0069371-T01-IM7" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	8195 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:8195] "GENIE-MSK-P-0038488-T01-IM6" "GENIE-MSK-P-0019305-T01-IM6" "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0069371-T01-IM7" ...
  .. .. ..$ mutation: num [1:8195] 2 0 2 2 3 1 4 2 1 2

In [5]:
create_pct <- function(run_list){
    pct_df <- list()
    i=1
    for(run in run_list){
        gene_order <- rownames(run$M$M$missense)
        sample_order <- colnames(run$M$M$missense)
        full_gam <-run$M$M$missense+run$M$M$truncating[gene_order,sample_order]
        full_gam[full_gam>1]<-1
        pct_df[[i]]<-data.frame('gene'=rownames(full_gam),'total'=rowSums(full_gam),'pct'=rowSums(full_gam)*100/ncol(full_gam))
        pct_df[[i]] <- pct_df[[i]] %>% mutate('sampling'=paste('Run',i,sep="_"))
        i=i+1
    }
    return(pct_df)
}

In [6]:
str(msk_p_sample_run_list[[1]]$M$M$missense)
str(msk_p_sample_run_list[[1]]$M$M$truncating)

 num [1:306, 1:8195] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  ..$ : chr [1:8195] "GENIE-MSK-P-0038488-T01-IM6" "GENIE-MSK-P-0019305-T01-IM6" "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0069371-T01-IM7" ...
 num [1:306, 1:8195] 0 0 0 0 0 0 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  ..$ : chr [1:8195] "GENIE-MSK-P-0038488-T01-IM6" "GENIE-MSK-P-0019305-T01-IM6" "GENIE-MSK-P-0014508-T01-IM6" "GENIE-MSK-P-0069371-T01-IM7" ...


In [7]:
msk_p_df <- create_pct(msk_p_sample_run_list)
msk_m_df <- create_pct(msk_m_sample_run_list)

In [8]:
str(msk_p_df)

List of 10
 $ :'data.frame':	306 obs. of  4 variables:
  ..$ gene    : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  ..$ total   : num [1:306] 62 194 14 12 2 91 20 11 79 3 ...
  ..$ pct     : num [1:306] 0.7566 2.3673 0.1708 0.1464 0.0244 ...
  ..$ sampling: chr [1:306] "Run_1" "Run_1" "Run_1" "Run_1" ...
 $ :'data.frame':	306 obs. of  4 variables:
  ..$ gene    : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  ..$ total   : num [1:306] 65 187 16 14 3 91 18 12 73 3 ...
  ..$ pct     : num [1:306] 0.7932 2.2819 0.1952 0.1708 0.0366 ...
  ..$ sampling: chr [1:306] "Run_2" "Run_2" "Run_2" "Run_2" ...
 $ :'data.frame':	306 obs. of  4 variables:
  ..$ gene    : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK1" ...
  ..$ total   : num [1:306] 75 191 18 22 2 89 20 18 78 2 ...
  ..$ pct     : num [1:306] 0.9152 2.3307 0.2196 0.2685 0.0244 ...
  ..$ sampling: chr [1:306] "Run_3" "Run_3" "Run_3" "Run_3" ...
 $ :'data.frame':	306 obs. of  4 variables:
  ..$ gene    : chr [1:306] "MTOR" "NRAS" "RIT1" "NTRK

In [9]:
msk_p_df_all <- bind_rows(msk_p_df)
rownames(msk_p_df_all)<-c(1:nrow(msk_p_df_all))
msk_m_df_all <- bind_rows(msk_m_df)
rownames(msk_m_df_all)<-c(1:nrow(msk_m_df_all))

In [10]:
head(msk_p_df_all)

,gene,total,pct,sampling
,<chr>,<dbl>,<dbl>,<chr>
1,MTOR,62,0.75655888,Run_1
2,NRAS,194,2.36729713,Run_1
3,RIT1,14,0.17083588,Run_1
4,NTRK1,12,0.14643075,Run_1
5,SDHC,2,0.02440513,Run_1
6,SPEN,91,1.11043319,Run_1


In [11]:
msk_p_df <- msk_p_df_all %>% pivot_wider(id_cols = gene,names_from = sampling,values_from = total) 
msk_m_df <- msk_m_df_all %>% pivot_wider(id_cols = gene,names_from = sampling,values_from = total) 

In [12]:
msk_p_df$average <- rowMeans(msk_p_df[, grep("^Run_", names(msk_p_df))])
msk_m_df$average <- rowMeans(msk_m_df[, grep("^Run_", names(msk_m_df))])

In [13]:
head(msk_m_df)
head(msk_p_df)

gene,Run_1,Run_2,Run_3,Run_4,Run_5,Run_6,Run_7,Run_8,Run_9,Run_10,average
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MTOR,59,60,61,55,57,57,57,55,62,54,57.7
NRAS,186,181,207,187,207,202,183,191,198,211,195.3
RIT1,17,22,15,17,20,18,19,19,15,18,18.0
NTRK1,21,21,17,17,19,14,16,20,20,18,18.3
SDHC,5,4,6,8,5,7,6,8,6,6,6.1
SPEN,77,90,75,71,81,74,85,81,92,84,81.0


gene,Run_1,Run_2,Run_3,Run_4,Run_5,Run_6,Run_7,Run_8,Run_9,Run_10,average
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MTOR,62,65,75,68,68,63,79,62,68,64,67.4
NRAS,194,187,191,198,195,188,189,201,186,191,192.0
RIT1,14,16,18,15,21,19,16,13,14,11,15.7
NTRK1,12,14,22,16,17,15,11,10,11,10,13.8
SDHC,2,3,2,5,3,3,1,0,2,4,2.5
SPEN,91,91,89,93,81,88,88,84,87,66,85.8


In [14]:
plot_df <- data.frame('gene'=msk_p_df$gene,'primary_pct'=round(msk_p_df$average,3),'met_pct'=round(msk_m_df$average,3))

In [15]:
plot_df <- na.omit(plot_df)

In [16]:
head(plot_df)

,gene,primary_pct,met_pct
,<chr>,<dbl>,<dbl>
1,MTOR,67.4,57.7
2,NRAS,192.0,195.3
3,RIT1,15.7,18.0
4,NTRK1,13.8,18.3
5,SDHC,2.5,6.1
6,SPEN,85.8,81.0


In [17]:
p_values <- list()
for(g in plot_df$gene){
    print(g)
    a<-unlist((unlist(msk_p_df %>% filter(gene==g) %>% select(grep("^Run_", names(msk_p_df))))))
    b<-unlist((unlist(msk_m_df %>% filter(gene==g) %>% select(grep("^Run_", names(msk_m_df))))))
    p_values[[g]]<- t.test(a,b,exact = FALSE)$p.value
}

[1] "MTOR"
[1] "NRAS"
[1] "RIT1"
[1] "NTRK1"
[1] "SDHC"
[1] "SPEN"
[1] "DDR2"
[1] "CDC73"
[1] "ELF3"
[1] "MDM4"
[1] "PARP1"
[1] "ID3"
[1] "FH"
[1] "AKT3"
[1] "TNFRSF14"
[1] "ARID1A"
[1] "SESN2"
[1] "CSF3R"
[1] "MPL"
[1] "MUTYH"
[1] "PIK3R3"
[1] "RAD54L"
[1] "CDKN2C"
[1] "JAK1"
[1] "FUBP1"
[1] "ERRFI1"
[1] "BCL10"
[1] "PIK3CD"
[1] "SUFU"
[1] "SHOC2"
[1] "TCF7L2"
[1] "FGFR2"
[1] "RET"
[1] "ARID5B"
[1] "TET1"
[1] "GATA3"
[1] "BMPR1A"
[1] "PTEN"
[1] "YAP1"
[1] "BIRC3"
[1] "ATM"
[1] "SDHD"
[1] "KMT2A"
[1] "CBL"
[1] "CHEK1"
[1] "RRAS2"
[1] "MYOD1"
[1] "WT1"
[1] "HRAS"
[1] "SDHAF2"
[1] "MEN1"
[1] "CCND1"
[1] "INPPL1"
[1] "EED"
[1] "SESN3"
[1] "SH2B3"
[1] "PTPN11"
[1] "TBX3"
[1] "ETV6"
[1] "RAB35"
[1] "HNF1A"
[1] "CDKN1B"
[1] "POLE"
[1] "RECQL"
[1] "KRAS"
[1] "KMT2D"
[1] "ERBB3"
[1] "GLI1"
[1] "CDK4"
[1] "LATS2"
[1] "FLT3"
[1] "BRCA2"
[1] "FOXO1"
[1] "RB1"
[1] "CYSLTR2"
[1] "DIS3"
[1] "AKT1"
[1] "NFKBIA"
[1] "NKX2-1"
[1] "FOXA1"
[1] "MAX"
[1] "RAD51B"
[1] "DICER1"
[1] "SPRED1"
[1] "KNSTRN"
[1]

In [18]:
plot_df$log10_primary<-log10(plot_df$primary_pct+1)
plot_df$log10_met<-log10(plot_df$met_pct+1)

In [19]:
plot_df<-plot_df %>% mutate('p_val'=unlist(p_values[gene]))

In [20]:
plot_df %>% arrange(desc(primary_pct))

gene,primary_pct,met_pct,log10_primary,log10_met,p_val
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TP53,3742.2,3934.7,3.573243,3.595022,2.152602e-12
KRAS,1711.3,1763.0,3.233580,3.246499,7.665606e-06
PIK3CA,1130.8,1122.8,3.053770,3.050689,2.885436e-01
APC,789.4,786.6,2.897847,2.896306,4.840429e-01
ARID1A,537.9,606.3,2.731508,2.783403,1.343183e-09
PTEN,437.1,444.7,2.641573,2.649043,2.830290e-01
BRAF,392.1,380.8,2.594503,2.581836,1.239725e-02
EGFR,351.0,373.0,2.546543,2.572872,1.028680e-04
RB1,325.1,348.8,2.513351,2.543820,1.057453e-05


In [21]:
plot_df<-na.omit(plot_df)

In [22]:
plot_df$qval <- p.adjust(plot_df$p_val) # p value FDR correction

In [23]:
head(plot_df)

,gene,primary_pct,met_pct,log10_primary,log10_met,p_val,qval
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,MTOR,67.4,57.7,1.835056,1.7686381,3.019830e-04,0.064926337
2,NRAS,192.0,195.3,2.285557,2.2929203,4.051999e-01,1.000000000
3,RIT1,15.7,18.0,1.222716,1.2787536,6.538795e-02,1.000000000
4,NTRK1,13.8,18.3,1.170262,1.2855573,6.239557e-03,1.000000000
5,SDHC,2.5,6.1,0.544068,0.8512583,1.422821e-05,0.003613966
6,SPEN,85.8,81.0,1.938520,1.9138139,1.619371e-01,1.000000000


In [24]:
plot_df$ratio <- (plot_df$met_pct+1)/(plot_df$primary_pct+1)

In [25]:
plot_df$log_pval<- -1*log10(plot_df$p_val)
plot_df$log_FDR<- -1*log10(plot_df$qval)
plot_df$log_ratio<- log2(plot_df$ratio)

In [26]:
plot_df<-plot_df %>% mutate(category=case_when( (qval<=0.05 & primary_pct>met_pct) ~ 'blue',
                                       (qval<=0.05 & primary_pct<met_pct) ~ 'red',
                                       TRUE ~ 'grey'))

In [27]:
head(plot_df)

,gene,primary_pct,met_pct,log10_primary,log10_met,p_val,qval,ratio,log_pval,log_FDR,log_ratio,category
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,MTOR,67.4,57.7,1.835056,1.7686381,3.019830e-04,0.064926337,0.8581871,3.5200176,1.187579,-0.22063582,grey
2,NRAS,192.0,195.3,2.285557,2.2929203,4.051999e-01,1.000000000,1.0170984,0.3923306,0.000000,0.02445933,grey
3,RIT1,15.7,18.0,1.222716,1.2787536,6.538795e-02,1.000000000,1.1377246,1.1845023,0.000000,0.18615132,grey
4,NTRK1,13.8,18.3,1.170262,1.2855573,6.239557e-03,1.000000000,1.3040541,2.2048463,0.000000,0.38300367,grey
5,SDHC,2.5,6.1,0.544068,0.8512583,1.422821e-05,0.003613966,2.0285714,4.8468496,2.442016,1.02046410,red
6,SPEN,85.8,81.0,1.938520,1.9138139,1.619371e-01,1.000000000,0.9447005,0.7906537,0.000000,-0.08207113,grey


In [28]:
plot_df %>% count(category)

category,n
<chr>,<int>
blue,29
grey,218
red,57


In [29]:
dim(plot_df)

[1] 304  12

In [31]:
dim(plot_df)

[1] 304  12

In [ ]:
saveRDS(plot_df,file='../figures_data/msk_p_m_gene_volcano_plot_data.rds')
